In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import timedelta, date
import pandas as pd
import pickle

In [2]:
def getTodayIndex(year, month, day):
    keys = {'syear': year, 'smonth': month, 'sday': day} # 要傳給期貨交易所的 key
    r = requests.post("http://www.taifex.com.tw/chinese/3/3_1_1.asp", data = keys) # 抓取這個 key 回傳網頁的原始碼
    soup = BeautifulSoup(r.text , "lxml") # 把原始碼做整理
    soup_data = soup.select('table')[2].select('table')[1].select('td')
    return soup_data

In [3]:
#可以透過這個方式，知道抓下來的碼到底長什麼樣子，注意不要抓到例假日會跑出錯誤
getTodayIndex(2017,12,29)

[<td class="12bk"><div align="center">TX      
 											  </div></td>,
 <td class="12bk"><div align="center">201801  </div></td>,
 <td align="right" class="12bk">10593</td>,
 <td align="right" class="12bk">10644</td>,
 <td align="right" class="12bk">10576</td>,
 <td align="right" class="12bk">10636</td>,
 <td align="middle" class="12green">
 <font color="red">â²75</font>
 </td>,
 <td align="middle" class="12green">
 <font color="red">â² 0.71%</font>
 </td>,
 <td align="right" class="12bk">7435</td>,
 <td align="right" class="12bk">122636</td>,
 <td align="right" class="12bk">130071</td>,
 <td align="right" class="12bk">
 											10633
 											</td>,
 <td align="right" class="12bk">96246</td>,
 <td align="right" class="12bk">10635</td>,
 <td align="right" class="12bk">10637</td>,
 <td align="right" class="12bk">10862</td>,
 <td align="right" class="12bk">10308</td>,
 <td class="12bk"> <div align="center">TX      
 											  </div></td>,
 <td class="12bk"> <div align="cent

In [4]:
def getData(start):

    # 先把 DataFrame 架構寫好
    data = {'date':[], 'open':[], 'high':[], 'low':[], 'settlement':[], 'volume':[], 'close':[], 'open_int':[], 'close_best_bid':[], 'close_best_ask':[]}

    # 這邊匯入我們想要的資料的起始日期
    day = date(start[0], start[1], start[2])
    # 這邊把停止日期設置在今天
    today = date.today()
    # 起始日至今日的總日數
    span = ((today - day).days) + 1
    # 因為是每日資料，所以間隔設為一日
    interval = timedelta(days=1)

    # 多定義一個把爬下來的資料放進 DataFrame 的方法
    def addtoData(column, index):
        try:
            #因為想要把'\r\n\t\t\t\t\t\t\t\t\t\t\t-\r\n\t\t\t\t\t\t\t\t\t\t\t'變成0，且要取代為字串，所以用以下方法
            #P.S. replace()只有在字串後面可以使用
            data[column].append(int(soup_data[index].text.replace('\r\n\t\t\t\t\t\t\t\t\t\t\t-\r\n\t\t\t\t\t\t\t\t\t\t\t','0')))
        except Exception as e:
            print (e)
            data[column].append(np.nan)
     # 把整個 span 內的 soup_data 中的當日期指各項資料抓下來
    for _ in range(span):
        try:
            soup_data = getTodayIndex(day.year, day.month, day.day)
            data['date'].append(day)
            addtoData('open', 2)
            addtoData('high', 3)
            addtoData('low', 4)
            addtoData('settlement', 5)
            addtoData('volume', 8)
            addtoData('close', 11)
            addtoData('open_int', 12)
            addtoData('close_best_bid', 13)
            addtoData('close_best_ask', 14)
            print ('%d/%d/%d is loaded' %(day.year, day.month, day.day))
        except Exception as e:
            #print (e)
            print ('stock market was not open on %d/%d/%d' %(day.year, day.month, day.day))
        # 日期往後加一天
        day += interval

   # 把 data 放到 DataFrame 裡面
    df = pd.DataFrame(data)
    return df  

# 從 2017/12/20 開始爬

#因為是每個月的第三個星期三，所以2017年12月的期貨到期，在結算價欄位產生缺值
df = getData([2017,12,20])

print (df[['date','open','close','volume']])

2017/12/20 is loaded
2017/12/21 is loaded
2017/12/22 is loaded
stock market was not open on 2017/12/23
stock market was not open on 2017/12/24
2017/12/25 is loaded
2017/12/26 is loaded
2017/12/27 is loaded
2017/12/28 is loaded
2017/12/29 is loaded
stock market was not open on 2017/12/30
stock market was not open on 2017/12/31
stock market was not open on 2018/1/1
2018/1/2 is loaded
2018/1/3 is loaded
2018/1/4 is loaded
2018/1/5 is loaded
stock market was not open on 2018/1/6
stock market was not open on 2018/1/7
2018/1/8 is loaded
2018/1/9 is loaded
2018/1/10 is loaded
          date   open  close  volume
0   2017-12-20  10457      0   10865
1   2017-12-21  10492  10495   15150
2   2017-12-22  10498  10533    8853
3   2017-12-25  10549  10525    7129
4   2017-12-26  10540  10417    1515
5   2017-12-27  10398  10474   13272
6   2017-12-28  10495  10561    8666
7   2017-12-29  10593  10633    7435
8   2018-01-02  10629  10708    9987
9   2018-01-03  10748  10788   15370
10  2018-01-04  1

In [5]:
with open('TWfuture.pickle','wb') as f:
    pickle.dump(df, f)

In [6]:
pickle_in = open('TWfuture.pickle', 'rb')
df = pickle.load(pickle_in)
df

,close,close_best_ask,close_best_bid,date,high,low,open,open_int,settlement,volume
0,0,10498,10497,2017-12-20,10509,10457,10457,30811,10497,10865
1,10495,10497,10496,2017-12-21,10539,10475,10492,87835,10497,15150
2,10533,10535,10534,2017-12-22,10535,10473,10498,87459,10535,8853
3,10525,10527,10526,2017-12-25,10564,10520,10549,88085,10526,7129
4,10417,10417,10416,2017-12-26,10541,10406,10540,91544,10416,1515
5,10474,10477,10475,2017-12-27,10485,10393,10398,93768,10476,13272
6,10561,10562,10561,2017-12-28,10593,10494,10495,94872,10562,8666
7,10633,10637,10635,2017-12-29,10644,10576,10593,96246,10636,7435
8,10708,10710,10709,2018-01-02,10710,10621,10629,103067,10709,9987
9,10788,10790,10789,2018-01-03,10797,10729,10748,102723,10789,15370
